In [5]:
import sys
import os

sys.path.append(os.path.abspath('..'))

print(f"Caminho '..' foi adicionado: {os.path.abspath('..')}")

Caminho '..' foi adicionado: c:\Users\luizz\Downloads\Entrega Pipeline Final


In [8]:
import pandas as pd
from sqlalchemy import create_engine
from database.config.database import get_connection_string
import glob
import os

# --- Configuração ---
# Define o caminho para a pasta gold
GOLD_DATA_DIR = "../data/gold/"
FILE_PATTERN = "*.csv"
full_path_pattern = os.path.join(GOLD_DATA_DIR, FILE_PATTERN)

# --- Contadores para resumo ---
files_loaded_count = 0
files_failed = []

print("Iniciando script de carregamento da camada GOLD para o Data Warehouse...")

try:
    # 1. Obter a string de conexão
    db_url = get_connection_string()

    # 2. Criar o engine do SQLAlchemy
    engine = create_engine(db_url)
    print("Conexão com o banco de dados estabelecida.")

    # 3. Encontrar todos os arquivos CSV na pasta gold
    csv_files = glob.glob(full_path_pattern)

    if not csv_files:
        print(f"Aviso: Nenhum arquivo .csv encontrado em '{full_path_pattern}'")
    else:
        print(f"Encontrados {len(csv_files)} arquivos CSV para carregar.")

    # 4. Iterar sobre cada arquivo CSV encontrado
    for file_path in csv_files:
        file_name = os.path.basename(file_path)
        
        try:
            # Extrair o nome da tabela do nome do arquivo
            # (ex: ../data/gold/minha_tabela.csv -> minha_tabela)
            table_name = os.path.splitext(file_name)[0]

            print(f"\nIniciando carregamento de '{file_name}' para a tabela '{table_name}'...")

            # Ler o CSV para um DataFrame
            df = pd.read_csv(file_path)

            if df.empty:
                print(f" -> Aviso: O arquivo '{file_name}' está vazio. Pulando.")
                continue

            # Salvar no PostgreSQL usando pandas
            # 'if_exists="replace"' garante que a tabela seja recriada a cada execução
            df.to_sql(
                name=table_name,
                con=engine,
                if_exists="replace",
                index=False
            )

            print(f" -> Sucesso! Dados de '{file_name}' carregados na tabela '{table_name}'.")
            files_loaded_count += 1

        except pd.errors.EmptyDataError:
            print(f" -> Erro: O arquivo '{file_name}' está vazio e não pôde ser lido. Pulando.")
            files_failed.append(file_name)
        except Exception as e_file:
            print(f" -> Erro ao processar o arquivo '{file_name}': {e_file}")
            files_failed.append(file_name)

    # 5. Imprimir resumo final
    print("\n--- Resumo do Carregamento ---")
    print(f"Arquivos carregados com sucesso: {files_loaded_count}")
    if files_failed:
        print(f"Arquivos que falharam: {len(files_failed)}")
        for f_name in files_failed:
            print(f"  - {f_name}")
    else:
        print("Todos os arquivos foram processados sem falhas.")
    print("---------------------------------")


except ImportError:
    print("Erro: SQLAlchemy não está instalado. Por favor, instale com 'pip install sqlalchemy psycopg2-binary'")
except Exception as e_global:
    print(f"Ocorreu um erro crítico durante a execução: {e_global}")

Iniciando script de carregamento da camada GOLD para o Data Warehouse...
Conexão com o banco de dados estabelecida.
Encontrados 7 arquivos CSV para carregar.

Iniciando carregamento de 'analise_sazonal_clima.csv' para a tabela 'analise_sazonal_clima'...
 -> Sucesso! Dados de 'analise_sazonal_clima.csv' carregados na tabela 'analise_sazonal_clima'.

Iniciando carregamento de 'benchmark_regional_rendimento.csv' para a tabela 'benchmark_regional_rendimento'...
 -> Sucesso! Dados de 'benchmark_regional_rendimento.csv' carregados na tabela 'benchmark_regional_rendimento'.

Iniciando carregamento de 'desempenho_regiao_cultura.csv' para a tabela 'desempenho_regiao_cultura'...
 -> Sucesso! Dados de 'desempenho_regiao_cultura.csv' carregados na tabela 'desempenho_regiao_cultura'.

Iniciando carregamento de 'perfil_climatico_regiao_cultura.csv' para a tabela 'perfil_climatico_regiao_cultura'...
 -> Sucesso! Dados de 'perfil_climatico_regiao_cultura.csv' carregados na tabela 'perfil_climatico_reg